# Prediction Error Filters

This notebook implements the most basic prediction error filter, which is constructed to predict observed training data $d_i^\text{obs}$ for $i=0, ..., N-1$ using $n$ earlier data $d_{i-j-1}^\text{obs}$ for $j=0, ..., n-1$ via the linear relation (discrete convolution)

\begin{equation}
d_i = \sum_{j=0}^{n-1} d_{i-j-1}^\text{obs} m_j\,.
\end{equation}

The task of finding the $n$ filter coefficients $m_j$ can be formulated as an inverse problem. Assuming that the observational errors are Gaussian with covariance matrix $\mathbf{C}$, the maximum-likelihood set of coefficients minimises the least-squares misfit functional

\begin{equation}
\chi(\mathbf{m}) = \frac{1}{2} \sum_{i,j=0}^{N-1} (d_i - d_i^\text{obs}) C_{ij}^{-1} (d_j - d_j^\text{obs})\,.
\end{equation}

Inserting the forward modelling equation and forcing the derivative of $\chi$ to zero, yields the normal equations for the maximum-likelihood coefficients $\hat{\mathbf{m}}$,

\begin{equation}
\sum_{i,j=0}^{N-1} d_i^\text{obs} C_{ij}^{-1} d_{j-q-1}^\text{obs} = \sum_{k=0}^{n-1} \left( \sum_{i,j=0}^{N-1} d_{i-k-1}^\text{obs} C_{ij}^{-1} d_{j-q-1}^\text{obs} \right)\,\hat{m}_k\,.
\end{equation}

This can be written in vector-matrix form,

\begin{equation}
b_q = \sum_{k=0}^{n-1} A_{qk} \hat{m}_k\,,
\end{equation}

with

\begin{equation}
b_q = \sum_{i,j=0}^{N-1} d_i^\text{obs} C_{ij}^{-1} d_{j-q-1}^\text{obs}\,,\qquad A_{qk}=\sum_{i,j=0}^{N-1} d_{i-k-1}^\text{obs} C_{ij}^{-1} d_{j-q-1}^\text{obs}\,.
\end{equation}

From this we see that the left-hand side and the matrix contain auto-correlations of the observations. Hence, linear predictions are made on the basis of the correlation properties of the data.

The data needed for this notebook can be downloaded with this Polybox link: https://polybox.ethz.ch/index.php/s/a3EDhinFdATu4qt .

# 0. Packages and setup

Import the necessary Python packages and add a few lines to embellish figures.

In [ ]:
import obspy
import numpy as np
import matplotlib.pyplot as plt
from obspy.signal.filter import bandpass

plt.rcParams["font.family"] = "Times"
plt.rcParams.update({'font.size': 65})
plt.rcParams['xtick.major.pad']='12'
plt.rcParams['ytick.major.pad']='12'

# 1. General input

Basic input, including the file to be read and the average spacing between channels.

In [ ]:
# GRIMSVÖTN DATA ========================================

# Input file.
input_file='/Users/andreas/Desktop/PEF/2021-05-26T00_02_14.998000Z.npy'
# Scale for plotting.
scale=4
# Receiver spacing.
dx=8.0
# Time increment.
dt=0.01

In [ ]:
# Minimun and maximum trace indices for plotting.
ix_min=150
ix_max=500

# Minimum time index and length of dataset for plotting.
i0=3900
Nplot=400

# 2. Data reading and plotting

## 2.1. Read and plot complete data

In [ ]:
cct=np.load(input_file)
print(cct.shape)

In [ ]:
nt=cct.shape[1]
nx=cct.shape[0]-1

t=np.linspace(0.0,nt*dt,nt)

In [ ]:
# Trace normalisation for plotting.
normalisation=200.0

plt.figure(figsize=(30,30))

for i in np.arange(ix_min,ix_max):
        
    data = cct[i,i0:i0+Nplot]/normalisation     
    dist_var = (i-1)*dx
    
    plt.plot(t[i0:i0+Nplot],(scale*data)+dist_var,'k-', alpha = 0.4)
    plt.fill_between(t[i0:i0+Nplot],(scale*data)+dist_var,y2=np.ones(np.shape(t[i0:i0+Nplot]))*dist_var,where=(data+dist_var>=dist_var), interpolate=True,fc='k',alpha=0.8)

plt.xlabel('time [s]')
plt.ylabel('distance [m]')
plt.title('raw data',pad=30)
plt.grid()
plt.show()

## 2.2. Filtering and downsampling

In [ ]:
# Minimum and maximum frequencies [Hz].
freqmin=1.0
freqmax=49.0
# Downsampling factor.
downsample=1

In [ ]:
# Frequency-domain filtering.
cct_filt=np.zeros(np.shape(cct))
for i in range(cct.shape[0]-1): cct_filt[i,:]=bandpass(cct[i,:],freqmin=freqmin,freqmax=freqmax,df=1.0/dt,corners=4,zerophase=True)

In [ ]:
# Downsampling.
cct_filt_down=cct_filt[:,0:nt:downsample]

nx=cct_filt_down.shape[0]-1
nt=cct_filt_down.shape[1]
dt=downsample*dt

t=np.linspace(0.0,nt*dt,nt)

i0=int(i0/downsample)
Nplot=int(Nplot/downsample)

In [ ]:
# Trace normalisation for plotting.
normalisation=200.0

plt.figure(figsize=(30,30))

for i in np.arange(ix_min,ix_max):
        
    data = cct_filt_down[i,i0:i0+Nplot]/normalisation     
    dist_var = (i-1)*dx
    
    plt.plot(t[i0:i0+Nplot],(scale*data)+dist_var,'k-', alpha = 0.4)
    plt.fill_between(t[i0:i0+Nplot],(scale*data)+dist_var,y2=np.ones(np.shape(t[i0:i0+Nplot]))*dist_var,where=(data+dist_var>=dist_var), interpolate=True,fc='k',alpha=0.8)

plt.xlabel('time [s]',labelpad=20)
plt.ylabel('distance [m]',labelpad=20)
plt.title('filtered data',pad=30)
plt.grid()
plt.tight_layout()
filename='OUTPUT/filtered_data.png'
plt.savefig(filename,dpi=200)
plt.show()

## 2.3. Pick a specific trace

As data, we first pick one of the available traces.

In [ ]:
# Index of the trace.
i_trace=340
# Pick trace.
d=cct_filt_down[i_trace,:]

# Plot individual trace.
plt.figure(figsize=(30,10))
plt.plot(d,'k')
plt.grid()
plt.xlabel('sample index')
plt.show()

# 3. Covariance matrix

We first need an estimate of the data covariance matrix $\mathbf{C}$. For this, we first need to determine the size of the training dataset $N$. We then take a window of this length and slide it across the noise part of the signal, prior to the first wave arrivals. From this, we get average cross-correlations between pairs of samples.

In [ ]:
# Number of data points in the training dataset.
Nd=100

In [ ]:
# Correlation vector.
corr=np.zeros(Nd)

# Minimum and maximum indices from the noise time series.
imin=100
imax=3900

# Compute mean and remove from data.
mean=np.mean(d[imin:imax])
d-=mean

# Compute correlation vector.
for i in np.arange(imin,imax):
    for j in range(Nd):
        corr[j]+=d[i]*d[i+j]

corr/=np.float(imax-imin)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(corr,'k')
plt.grid()
plt.title('average cross-correlation',pad=20)
plt.xlabel('sample index')
plt.xlim([0,Nd-1])
plt.show()

We finally build $\mathbf{C}$ from the average cross-correlation by putting shifted copies of it into the rows of the matrix.

In [ ]:
C=np.zeros([Nd,Nd])
for i in range(Nd): C[i,:]=np.roll(corr,i)
Cinv=np.linalg.inv(C)

# 4. Prediction

We start the simple prediction step by choosing a part of the time series that contains actual signal. Based on this, we build the vector $\mathbf{b}$ and the matrix $\mathbf{A}$.

In [ ]:
# Number of filter coefficients (model parameters)
n=88

In [ ]:
# Starting index of the time series of interest (training dataset).
i0=4100

# Plot the training dataset.
plt.figure(figsize=(30,10))
plt.plot(np.arange(-n,Nd),d[i0-n:i0+Nd],'--k')
plt.plot(np.arange(-n,Nd),d[i0-n:i0+Nd],'ko',MarkerSize=12)
plt.plot(np.arange(0,Nd),d[i0:i0+Nd],'k',LineWidth=2)
plt.grid()
plt.xlim([-n,Nd])
plt.title('training dataset',pad=20)
plt.xlabel('sample index')
plt.show()

In the next step, we assemble $\mathbf{b}$ and $\mathbf{A}$. To make this more efficient, we use the fact that the matrix $X_{i,q}=\sum_{j=0}^{N} C_{ij}^{-1} d_{j-q-1}^\text{obs}$ appears in both $\mathbf{b}$ and $\mathbf{A}$.

In [ ]:
# Build the auxiliary matrix X.
X=np.zeros([Nd,n])
for q in range(n): X[:,q]=np.dot(Cinv,d[(i0-q-1):(i0-q-1+Nd)])
    
# Build vector b.
b=np.dot(d[i0:(i0+Nd)],X)

# Build matrix A.
A=np.zeros([n,n])
for k in range(n): A[:,k]=np.dot(d[(i0-k-1):(i0-k-1+Nd)],X)

In [ ]:
# Solve linear system.
p=np.dot(np.linalg.inv(A),b)
print(p)

# Plot filter coefficients.
plt.figure(figsize=(30,10))
plt.plot(p,'k',LineWidth=2)
plt.plot(p,'ko',MarkerSize=12)
plt.grid()
plt.xlabel('coefficient index i')
plt.ylabel('filter coefficient')
plt.show()

## 4.1. Predicting the training dataset

Trying to predict the training dataset helps to check if the algorithm works at all, and it allows us to choose the proper number of model parameters that roughly produces an rms error of 1.

In [ ]:
# Compute the prediction.
d_pred=np.zeros(Nd)

for j in range(Nd):
    for i in range(n):
        d_pred[j]+=d[i0+j-i-1]*p[i]

In [ ]:
# Plot the comparison of observed and predicted data.
plt.figure(figsize=(30,15))
plt.plot(np.arange(-n,Nd),d[i0-n:i0+Nd],'--k')
plt.plot(np.arange(-n,Nd),d[i0-n:i0+Nd],'ko',MarkerSize=12)
plt.plot(np.arange(0,Nd),d[i0:i0+Nd],'k',LineWidth=4)
plt.plot(np.arange(0,Nd),d_pred,c=[0.75,0.75,0.75],LineWidth=4)
plt.plot(np.arange(0,Nd),d_pred,'o',c=[0.5,0.5,0.5],LineWidth=1,MarkerSize=12)
plt.grid()
plt.xlim([-n-1,Nd])
plt.title('observation/prediction comparison',pad=30)
plt.xlabel('sample index',labelpad=20)
plt.ylabel('nanostrain / s', labelpad=20)
plt.tight_layout()
filename='OUTPUT/training_data_prediction.png'
plt.savefig(filename,dpi=200)
plt.show()

plt.figure(figsize=(30,10))
plt.plot(np.arange(0,Nd),d[i0:i0+Nd]-d_pred,'k',LineWidth=2)
plt.plot(np.arange(0,Nd),d[i0:i0+Nd]-d_pred,'ko',MarkerSize=12)
plt.grid()
plt.xlim([-n-1,Nd])
plt.title('difference observation - prediction',pad=30)
plt.xlabel('sample index', labelpad=20)
plt.ylabel('nanostrain / s', labelpad=20)
plt.show()

To check how good the prediction actually is, we compute the rms error using the previously estimated covariance matrix $\mathbf{C}$.

In [ ]:
# Compute residual.
res=np.dot(d_pred-d[i0:(i0+Nd)],Cinv)
res=np.dot(res,d_pred-d[i0:(i0+Nd)])
res=np.sqrt(res/np.float(Nd))
print('rms error: %f' % res)

## 4.2. Predicting beyond the training dataset

To see how useful the model parameters are to predict more than just the training dataset, we extend this time series to more samples.

In [ ]:
# New number of samples.
Nd_new=2*Nd

# Compute the prediction.
d_pred=np.zeros(Nd_new)

for j in range(Nd_new):
    for i in range(n):
        d_pred[j]+=d[i0+j-i-1]*p[i]

In [ ]:
# Plot the comparison of observed and predicted data.
plt.figure(figsize=(40,20))
plt.plot(np.arange(-n,Nd_new),d[i0-n:i0+Nd_new],'--k')
plt.plot(np.arange(-n,Nd_new),d[i0-n:i0+Nd_new],'ko',MarkerSize=12)
plt.plot(np.arange(0,Nd_new),d[i0:i0+Nd_new],'k',LineWidth=2)
plt.plot(np.arange(0,Nd_new),d_pred,c=[0.75,0.75,0.75],LineWidth=2)
plt.plot(np.arange(0,Nd_new),d_pred,'o',c=[0.75,0.75,0.75],MarkerSize=12)
plt.grid()
plt.xlim([-n,Nd_new])
plt.title('observation/prediction comparison',pad=30)
plt.xlabel('sample index',labelpad=20)
plt.ylabel('nanostrain / s', labelpad=20)
plt.tight_layout()
filename='OUTPUT/beyond_training_data_prediction.png'
plt.savefig(filename,dpi=200)
plt.show()

plt.figure(figsize=(40,20))
plt.plot(np.arange(0,Nd_new),d[i0:i0+Nd_new]-d_pred,'k',LineWidth=2)
plt.plot(np.arange(0,Nd_new),d[i0:i0+Nd_new]-d_pred,'ko',MarkerSize=12)
plt.grid()
plt.xlim([-n,Nd_new])
plt.title('difference observation - prediction',pad=30)
plt.xlabel('sample index')
plt.show()

In [ ]:
# Correlation vector.
corr_new=np.zeros(Nd_new)

# Minimum and maximum indices from the noise time series.
imin=100
imax=3900

# Compute correlation vector.
for i in np.arange(imin,imax):
    for j in range(Nd_new):
        corr_new[j]+=d[i]*d[i+j]

corr_new/=np.float(imax-imin)
 
# Make the new correlation matrix.
C_new=np.zeros([Nd_new,Nd_new])
for i in range(Nd_new): C_new[i,:]=np.roll(corr_new,i)
Cinv_new=np.linalg.inv(C_new)

# Compute residual.
res=np.dot(d_pred-d[i0:(i0+Nd_new)],Cinv_new)
res=np.dot(res,d_pred-d[i0:(i0+Nd_new)])
res=np.sqrt(res/np.float(Nd_new))
print('rms error: %f' % res)

## 4.3. Predicting a different time series

Now we use the same model parameters but for predicting a different time series.

In [ ]:
# Index of the trace.
i_trace_new=350
# Pick trace.
d_new=cct_filt_down[i_trace_new,:]

# Plot individual trace.
plt.figure(figsize=(30,10))
plt.plot(d_new,'k')
plt.grid()
plt.xlabel('sample index')
plt.show()

In [ ]:
# Compute the prediction.
d_pred=np.zeros(Nd_new)

for j in range(Nd_new):
    for i in range(n):
        d_pred[j]+=d_new[i0+j-i-1]*p[i]

In [ ]:
# Plot the comparison of observed and predicted data.
plt.figure(figsize=(40,20))
plt.plot(np.arange(-n,Nd_new),d_new[i0-n:i0+Nd_new],'--k')
plt.plot(np.arange(-n,Nd_new),d_new[i0-n:i0+Nd_new],'ko',MarkerSize=12)
plt.plot(np.arange(0,Nd_new),d_new[i0:i0+Nd_new],'k',LineWidth=2)
plt.plot(np.arange(0,Nd_new),d_pred,c=[0.75,0.75,0.75],LineWidth=2)
plt.plot(np.arange(0,Nd_new),d_pred,'o',c=[0.75,0.75,0.75],MarkerSize=12)
plt.grid()
plt.xlim([-n,Nd_new])
plt.title('observation/prediction comparison',pad=30)
plt.xlabel('sample index',labelpad=20)
plt.ylabel('nanostrain / s', labelpad=20)
plt.tight_layout()
filename='OUTPUT/other_data_prediction.png'
plt.savefig(filename,dpi=200)
plt.show()

plt.figure(figsize=(40,20))
plt.plot(np.arange(0,Nd_new),d_new[i0:i0+Nd_new]-d_pred,'k',LineWidth=2)
plt.plot(np.arange(0,Nd_new),d_new[i0:i0+Nd_new]-d_pred,'ko',MarkerSize=12)
plt.grid()
plt.xlim([-n,Nd_new])
plt.title('difference observation - prediction',pad=30)
plt.xlabel('sample index')
plt.show()

In [ ]:
# Correlation vector.
corr_new=np.zeros(Nd_new)

# Minimum and maximum indices from the noise time series.
imin=100
imax=3900

# Compute correlation vector.
for i in np.arange(imin,imax):
    for j in range(Nd_new):
        corr_new[j]+=d_new[i]*d_new[i+j]

corr_new/=np.float(imax-imin)
 
# Make the new correlation matrix.
C_new=np.zeros([Nd_new,Nd_new])
for i in range(Nd_new): C_new[i,:]=np.roll(corr_new,i)
Cinv_new=np.linalg.inv(C_new)

# Compute residual.
res=np.dot(d_pred-d_new[i0:(i0+Nd_new)],Cinv_new)
res=np.dot(res,d_pred-d_new[i0:(i0+Nd_new)])
res=np.sqrt(res/np.float(Nd_new))
print('rms error: %f' % res)